In [1]:
# импортируем необходимые библиотеки
import xgboost
import numpy as np
import pandas as pd

In [2]:
# загружаем данные для классификации
data = pd.read_csv('Data/xgboost_classification.csv', sep=';')
# выводим наблюдения датафрейма
data

,age,response
0,23,0
1,45,0
2,44,0
3,20,0
4,49,0
5,56,1
6,55,1
7,43,1
8,23,0
9,67,0


In [3]:
# создаем массив меток зависимой переменной
y = data.pop('response')
# создаем массив признаков
X = data

In [4]:
# создаем объект DMatrix для обучения
dtrain = xgboost.DMatrix(data=X, label=y)

In [5]:
# задаем значения параметров и гиперпараметров
params = {'max_depth': 1, 
          'eta': 1.0, 
          'objective': 'binary:logistic',
          'lambda': 0, 
          'base_score': 0.5}

In [6]:
# обучаем модель XGBoost с одной итерацией
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
# записываем спрогнозированные вероятности положительного класса 
# и информацию о модели (информацию о весах листьев)
pr = model.predict(dtrain)
model_info = model.get_dump()[0]
# смотрим вероятности и веса листьев
print(pr, '\n\n', model_info)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0.28224894 0.28224894 0.28224894 0.28224894 0.28224894 0.59868765
 0.59868765 0.28224894 0.28224894 0.59868765 0.28224894 0.28224894
 0.28224894 0.28224894 0.28224894 0.28224894 0.28224894 0.59868765
 0.59868765 0.28224894] 

 0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.933333337
	2:leaf=0.400000006



In [7]:
# создаем функцию, которое выполняет преобразование
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# записываем глобальное смещение
base_score = 0.5
# записываем вес левого узла
leaf_weight_for_less_than_55 = -0.933333337
# записываем вес правого узла
leaf_weight_for_55_and_more = 0.400000006

# вычисляем вероятность положительного класса 
# для наблюдений, в которых age < 55
prediction_for_less_than_55 = sigmoid(
    np.log(base_score / (1 - base_score)) + leaf_weight_for_less_than_55)
# вычисляем вероятность положительного класса
# для наблюдений, в которых age >= 55
prediction_for_55_and_more = sigmoid(
    np.log(base_score / (1 - base_score)) + leaf_weight_for_55_and_more)
# печатаем вероятность положительного класса 
# для наблюдений, в которых age < 55
print('вероятность класса 1 для age <55', prediction_for_less_than_55)
# печатаем вероятность положительного класса 
# для наблюдений, в которых age >= 55
print('вероятность класса 1 для age >=55', prediction_for_55_and_more)

вероятность класса 1 для age <55 0.28224894229945025
вероятность класса 1 для age >=55 0.5986876615540165


In [8]:
# для удобства создадим датафрейм, в который запишем 
# константные прогнозы, значения градиентов и гессианов
df = pd.concat([X, y], axis=1)
# записываем массив константных прогнозов, 
# они равны base_score = 0.5
preds = np.array([0.5] * 20)
# вычисляем градиенты
grad = preds - df['response']
# вычисляем гессианы
hess = preds * (1 - preds)
# записываем константные прогнозы, градиенты
# и гессианы в датафрейм
df['preds'] = preds
df['grad'] = grad
df['hess'] = hess
# выводим датафрейм
df

,age,response,preds,grad,hess
0,23,0,0.5,0.5,0.25
1,45,0,0.5,0.5,0.25
2,44,0,0.5,0.5,0.25
3,20,0,0.5,0.5,0.25
4,49,0,0.5,0.5,0.25
5,56,1,0.5,-0.5,0.25
6,55,1,0.5,-0.5,0.25
7,43,1,0.5,-0.5,0.25
8,23,0,0.5,0.5,0.25
9,67,0,0.5,0.5,0.25


In [9]:
# записываем eta
eta = 1.0
# записываем lambda
lambda_ = 0
# вычисляем сумму градиентов в левом узле дерева 1
G_left = df[df['age'] < 55].sum()['grad']
# вычисляем сумму гессианов в левом узле дерева 1
H_left = df[df['age'] < 55].sum()['hess']
# вычисляем вес левого узла дерева 1 как сумму градиентов
# в левом узле, поделенную на сумму гессианов в
# левом узле плюс lambda
weight_left = eta * (- (G_left / (H_left + lambda_)))
print('сумма градиентов в левом узле дерева 1', G_left)
print('сумма гессианов в левом узле дерева 1', H_left)
print('вес левого узла дерева 1', round(weight_left, 3))

сумма градиентов в левом узле дерева 1 3.5
сумма гессианов в левом узле дерева 1 3.75
вес левого узла дерева 1 -0.933


In [10]:
# вычисляем сумму градиентов в правом узле дерева 1
G_right = df[df['age'] >= 55].sum()['grad']
# вычисляем сумму гессианов в правом узле дерева 1
H_right = df[df['age'] >= 55].sum()['hess']
# вычисляем вес правого узла дерева 1 как сумму градиентов
# в правом узле, поделенную на сумму гессианов в
# правом узле плюс lambda
weight_right = eta * (- (G_right / (H_right + lambda_)))
print('сумма градиентов в правом узле дерева 1', G_right)
print('сумма гессианов в правом узле дерева 1', H_right)
print('вес правого узла дерева 1', round(weight_right, 3))

сумма градиентов в правом узле дерева 1 -0.5
сумма гессианов в правом узле дерева 1 1.25
вес правого узла дерева 1 0.4


In [11]:
model = xgboost.train(params=params, num_boost_round=2, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr)
print('')
for line in model.get_dump(): 
    print(line)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0.41287592 0.1725412  0.1725412  0.41287592 0.1725412  0.44166934
 0.44166934 0.41287592 0.41287592 0.44166934 0.1725412  0.41287592
 0.41287592 0.41287592 0.1725412  0.41287592 0.41287592 0.44166934
 0.44166934 0.41287592]

0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.933333337
	2:leaf=0.400000006

0:[age<44] yes=1,no=2,missing=1
	1:leaf=0.581244171
	2:leaf=-0.634389997



In [12]:
# записываем массив спрогнозированных вероятностей
preds = np.array([0.28224894, 0.28224894, 0.28224894, 0.28224894, 0.28224894, 0.59868765,
                  0.59868765, 0.28224894, 0.28224894, 0.59868765, 0.28224894, 0.28224894,
                  0.28224894, 0.28224894, 0.28224894, 0.28224894, 0.28224894, 0.59868765,
                  0.59868765, 0.28224894])

In [13]:
# для удобства вновь создадим датафрейм, в который запишем 
# спрогнозированные вероятности, значения градиентов и гессианов
df = pd.concat([X, y], axis=1)
# вычисляем градиенты
grad = preds - df['response']
# вычисляем гессианы
hess = preds * (1 - preds)
# записываем спрогнозированные вероятности, 
# градиенты и гессианы в датафрейм
df['preds'] = preds
df['grad'] = grad
df['hess'] = hess
# выводим датафрейм
df

,age,response,preds,grad,hess
0,23,0,0.282249,0.282249,0.202584
1,45,0,0.282249,0.282249,0.202584
2,44,0,0.282249,0.282249,0.202584
3,20,0,0.282249,0.282249,0.202584
4,49,0,0.282249,0.282249,0.202584
5,56,1,0.598688,-0.401312,0.240261
6,55,1,0.598688,-0.401312,0.240261
7,43,1,0.282249,-0.717751,0.202584
8,23,0,0.282249,0.282249,0.202584
9,67,0,0.598688,0.598688,0.240261


In [14]:
# записываем eta
eta = 1.0
# записываем lambda
lambda_ = 0
# вычисляем сумму градиентов в левом узле дерева 2
G_left = df[df['age'] < 44].sum()['grad']
# вычисляем сумму гессианов в левом узле дерева 2
H_left = df[df['age'] < 44].sum()['hess']
# вычисляем вес левого узла дерева 2 как сумму градиентов
# в левом узле, поделенную на сумму гессианов в
# левом узле плюс lambda
weight_left = eta * (- (G_left / (H_left + lambda_)))
print('сумма градиентов в левом узле дерева 2', G_left)
print('сумма гессианов в левом узле дерева 2', H_left)
print('вес левого узла дерева 2', round(weight_left, 3))

сумма градиентов в левом узле дерева 2 -1.1775106
сумма гессианов в левом узле дерева 2 2.025844758688764
вес левого узла дерева 2 0.581


In [15]:
# вычисляем сумму градиентов в правом узле дерева 2
G_right = df[df['age'] >= 44].sum()['grad']
# вычисляем сумму гессианов в правом узле дерева 2
H_right = df[df['age'] >= 44].sum()['hess']
# вычисляем вес правого узла дерева 2 как сумму градиентов
# в правом узле, поделенную на сумму гессианов в
# правом узле плюс lambda
weight_right = eta * (- (G_right / (H_right + lambda_)))
print('сумма градиентов в правом узле дерева 2', G_right)
print('сумма гессианов в правом узле дерева 2', H_right)
print('вес правого узла дерева 2', round(weight_right, 3))

сумма градиентов в правом узле дерева 2 1.4046829500000002
сумма гессианов в правом узле дерева 2 2.2142261180317693
вес правого узла дерева 2 -0.634


In [16]:
base_score = 0.5
leaf_weight_for_less_than_55 = -0.933333337
leaf_weight_for_55_and_more = 0.400000006

leaf_weight_for_less_than_44 = 0.581244171
leaf_weight_for_44_and_more = -0.634389997

prediction_for_obs_0 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_weight_for_less_than_55 + leaf_weight_for_less_than_44))
prediction_for_obs_1 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_weight_for_less_than_55 + leaf_weight_for_44_and_more))
print('вероятность класса 1 для наблюдения с индексом 0', prediction_for_obs_0)
print('вероятность класса 1 для наблюдения с индексом 1', prediction_for_obs_1)

вероятность класса 1 для наблюдения с индексом 0 0.4128758956142221
вероятность класса 1 для наблюдения с индексом 1 0.17254119071402557


In [17]:
params = {'max_depth': 1, 
          'eta': 1.0, 
          'objective': 'binary:logistic',
          'lambda': 0, 
          'base_score': 0.3}
model = xgboost.train(params=params, num_boost_round=2, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr)
for line in model.get_dump(): 
    print(line)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0.41786543 0.15039323 0.15039323 0.41786543 0.15039323 0.4640077
 0.4640077  0.41786543 0.41786543 0.4640077  0.15039323 0.41786543
 0.41786543 0.41786543 0.15039323 0.41786543 0.41786543 0.4640077
 0.4640077  0.41786543]
0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.158730209
	2:leaf=1.42857134

0:[age<44] yes=1,no=2,missing=1
	1:leaf=0.674485743
	2:leaf=-0.72549212



In [18]:
base_score = 0.3
leaf_weight_for_less_than_55 = -0.158730209
leaf_weight_for_55_and_more = 1.42857134

leaf_weight_for_less_than_44 = 0.674485743
leaf_weight_for_44_and_more = -0.72549212

prediction_for_obs_0 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_weight_for_less_than_55 + leaf_weight_for_less_than_44))
prediction_for_obs_1 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_weight_for_less_than_55 + leaf_weight_for_44_and_more))
print('вероятность класса 1 для наблюдения с индексом 0', prediction_for_obs_0)
print('вероятность класса 1 для наблюдения с индексом 1', prediction_for_obs_1)

вероятность класса 1 для наблюдения с индексом 0 0.4178653987797429
вероятность класса 1 для наблюдения с индексом 1 0.1503932341296568


In [19]:
params = {'max_depth': 1, 
          'eta': 0.5, 
          'objective': 'binary:logistic',
          'lambda': 0, 
          'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
model_info = model.get_dump()[0]
print(model_info)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.466666669
	2:leaf=0.200000003



In [20]:
params = {'max_depth': 1, 
          'eta': 0.5, 
          'objective': 'binary:logistic',
          'lambda': 1.0, 
          'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
model_info = model.get_dump()[0]
print(model_info)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.368421048
	2:leaf=0.111111112



In [21]:
df = pd.concat([X, y], axis=1)
preds = np.array([0.5] * 20)
grad = preds - df['response']
hess = preds * (1 - preds)
df['grad'] = grad
df['hess'] = hess
G_left = df[df['age'] < 55].sum()['grad']
H_left = df[df['age'] < 55].sum()['hess']
G_right = df[df['age'] >= 55].sum()['grad']
H_right = df[df['age'] >= 55].sum()['hess']
eta = 0.5
lambda_ = 1.0
weight_left_when_lambda_1 = eta * (- (G_left / (H_left + lambda_)))
weight_right_when_lambda_1 = eta * (- (G_right / (H_right + lambda_)))
print('вес левого узла при lambda 1', round(weight_left_when_lambda_1, 3))
print('вес правого узла при lambda 1', round(weight_right_when_lambda_1, 3))

вес левого узла при lambda 1 -0.368
вес правого узла при lambda 1 0.111


In [22]:
params = {'max_depth': 1, 
          'eta': 1.0, 
          'objective': 'binary:logistic',
          'lambda': 0, 
          'max_delta_step': 0, 
          'base_score': 0.5}
model_without_max_delta_step = xgboost.train(
    params=params, num_boost_round=1, dtrain=dtrain)

params = {'max_depth': 1, 
          'eta': 1.0, 
          'objective': 'binary:logistic',
          'lambda': 0, 
          'max_delta_step': 0.5, 
          'base_score': 0.5}
model_with_max_delta_step = xgboost.train(
    params=params, num_boost_round=1, dtrain=dtrain)

print(model_without_max_delta_step.get_dump()[0])
print(model_with_max_delta_step.get_dump()[0])

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.933333337
	2:leaf=0.400000006

0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.5
	2:leaf=0.400000006



In [23]:
# загружаем данные для регрессии
data = pd.read_csv('Data/xgboost_regression.csv', sep=';')
# выводим датафрейм
data

,age,days_of_delinq
0,23,6
1,45,8
2,44,10
3,20,12
4,49,15
5,56,10
6,55,14
7,43,20
8,23,11
9,67,6


In [24]:
# создаем массив меток зависимой переменной
y = data.pop('days_of_delinq')
# создаем массив признаков
X = data

In [25]:
# создаем объект DMatrix
dtrain = xgboost.DMatrix(data=X, label=y)

In [26]:
# задаем значения параметров и гиперпараметров
params = {'max_depth': 1, 
          'eta': 1.0, 
          'objective': 'reg:linear', 
          'lambda': 0, 
          'base_score': 0.5}

In [27]:
# обучаем модель XGBoost с одной итерацией
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
# записываем спрогнозированные значения и информацию 
# о модели (информацию о весах листьев)
pr = model.predict(dtrain)
model_info = model.get_dump()[0]
# смотрим спрогнозированные значения и веса листьев
print(pr, '\n\n', model_info)

[14:57:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12.375 12.375 12.375 12.375 12.375  6.    12.375 12.375 12.375  6.
 12.375 12.375 12.375 12.375 12.375 12.375 12.375  6.     6.    12.375] 

 0:[age<56] yes=1,no=2,missing=1
	1:leaf=11.875
	2:leaf=5.5



In [28]:
base_score = 0.5
leaf_weight_for_less_than_56 = 11.875
leaf_weight_for_56_and_more = 5.5
prediction_for_less_than_56 = base_score + leaf_weight_for_less_than_56
prediction_for_56_and_more = base_score + leaf_weight_for_56_and_more
print('прогноз для age <56', prediction_for_less_than_56)
print('прогноз для age >=56', prediction_for_56_and_more)

прогноз для age <56 12.375
прогноз для age >=56 6.0


In [29]:
# для удобства создадим датафрейм, в который запишем 
# константные прогнозы, значения градиентов и гессианов
df = pd.concat([X, y], axis=1)
# записываем массив константных прогнозов, 
# они равны base_score = 0.5
preds = np.array([0.5] * 20)
# вычисляем градиенты
grad = preds - df['days_of_delinq']
# вычисляем гессианы
hess = np.array([1] * 20)
# записываем константные прогнозы, градиенты
# и гессианы в датафрейм
df['preds'] = preds
df['grad'] = grad
df['hess'] = hess
# выводим датафрейм
df

,age,days_of_delinq,preds,grad,hess
0,23,6,0.5,-5.5,1
1,45,8,0.5,-7.5,1
2,44,10,0.5,-9.5,1
3,20,12,0.5,-11.5,1
4,49,15,0.5,-14.5,1
5,56,10,0.5,-9.5,1
6,55,14,0.5,-13.5,1
7,43,20,0.5,-19.5,1
8,23,11,0.5,-10.5,1
9,67,6,0.5,-5.5,1


In [30]:
# записываем eta
eta = 1.0
# записываем lambda
lambda_ = 0
# вычисляем сумму градиентов в левом узле дерева 1
G_left = df[df['age'] < 56].sum()['grad']
# вычисляем сумму гессианов в левом узле дерева 1
H_left = df[df['age'] < 56].sum()['hess']
# вычисляем вес левого узла дерева 1 как сумму градиентов
# в левом узле, поделенную на сумму гессианов в
# левом узле плюс lambda
weight_left = eta * (- (G_left / (H_left + lambda_)))
print('сумма градиентов в левом узле дерева 1', G_left)
print('сумма гессианов в левом узле дерева 1', H_left)
print('вес левого узла дерева 1', round(weight_left, 3))

сумма градиентов в левом узле дерева 1 -190.0
сумма гессианов в левом узле дерева 1 16.0
вес левого узла дерева 1 11.875


In [31]:
# вычисляем сумму градиентов в правом узле дерева 1
G_right = df[df['age'] >= 56].sum()['grad']
# вычисляем сумму гессианов в правом узле дерева 1
H_right = df[df['age'] >= 56].sum()['hess']
# вычисляем вес правого узла дерева 1 как сумму градиентов
# в правом узле, поделенную на сумму гессианов в
# правом узле плюс lambda
weight_right = eta * (- (G_right / (H_right + lambda_)))
print('сумма градиентов в правом узле дерева 1', G_right)
print('сумма гессианов в правом узле дерева 1', H_right)
print('вес правого узла дерева 1', round(weight_right, 3))

сумма градиентов в правом узле дерева 1 -22.0
сумма гессианов в правом узле дерева 1 4.0
вес правого узла дерева 1 5.5
